In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain_community langchain_openai chromadb

In [3]:
!git clone https://github.com/Prometheus-AI-Project/2024_2_Basic_Study_HairStyler.git /content/drive/MyDrive/2024_2_Basic_Study_HairStyler

fatal: destination path '/content/drive/MyDrive/2024_2_Basic_Study_HairStyler' already exists and is not an empty directory.


In [4]:
%cd /content/drive/MyDrive/2024_2_Basic_Study_HairStyler/

/content/drive/MyDrive/2024_2_Basic_Study_HairStyler


In [6]:
!pip install torch==2.0.0 torchvision==0.15.1 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118
!pip install ftfy regex tqdm matplotlib jupyter ipykernel opencv-python scikit-image kornia==0.6.7 face-alignment==1.3.5 dlib
!pip install git+https://github.com/openai/CLIP.git
!pip install -q streamlit

Looking in indexes: https://download.pytorch.org/whl/cu118
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-5_v782hf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-5_v782hf
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [7]:
# 밑에 import 오류나면 실행하기
import sys
sys.path.append('.')

In [8]:
!pip install torch

In [9]:
!rm -rf /root/.cache/torch_extensions/py311_cu118

In [10]:
!pip install Ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 7.3 MB/s eta 0:00:00


In [11]:
!rm -rf /tmp/torch_extensions

In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from scripts.Embedding import Embedding
from scripts.text_proxy import TextProxy
from scripts.ref_proxy import RefProxy
from scripts.sketch_proxy import SketchProxy
from scripts.bald_proxy import BaldProxy
from scripts.color_proxy import ColorProxy
from scripts.feature_blending import hairstyle_feature_blending
from utils.seg_utils import vis_seg
from utils.mask_ui import painting_mask
from utils.image_utils import display_image_list, process_display_input
from utils.model_utils import load_base_models
from utils.options import Options

In [13]:
opts = Options().parse(jupyter=True)
src_name = '168125'# source image name you want to edit

image_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
g_ema, mean_latent_code, seg = load_base_models(opts)
ii2s = Embedding(opts, g_ema, mean_latent_code[0,0])

if not os.path.isfile(os.path.join(opts.src_latent_dir, f"{src_name}.npz")):
    inverted_latent_w_plus, inverted_latent_F = ii2s.invert_image_in_FS(image_path=f'{opts.src_img_dir}/{src_name}.jpg')
    save_latent_path = os.path.join(opts.src_latent_dir, f'{src_name}.npz')
    np.savez(save_latent_path, latent_in=inverted_latent_w_plus.detach().cpu().numpy(),
                latent_F=inverted_latent_F.detach().cpu().numpy())

src_latent = torch.from_numpy(np.load(f'{opts.src_latent_dir}/{src_name}.npz')['latent_in']).cuda()
src_feature = torch.from_numpy(np.load(f'{opts.src_latent_dir}/{src_name}.npz')['latent_F']).cuda()
src_image = image_transform(Image.open(f'{opts.src_img_dir}/{src_name}.jpg').convert('RGB')).unsqueeze(0).cuda()
input_mask = torch.argmax(seg(src_image)[1], dim=1).long().clone().detach()

bald_proxy = BaldProxy(g_ema, opts.bald_path)
text_proxy = TextProxy(opts, g_ema, seg, mean_latent_code)
ref_proxy = RefProxy(opts, g_ema, seg, ii2s)
sketch_proxy = SketchProxy(g_ema, mean_latent_code, opts.sketch_path)
color_proxy = ColorProxy(opts, g_ema, seg)

edited_hairstyle_img = src_image
def hairstyle_editing(global_cond=None, local_sketch=False, paint_the_mask=False, \
                      src_latent=src_latent, src_feature=src_feature, input_mask=input_mask, src_image=src_image, \
                        latent_global=None, latent_local=None, latent_bald=None, local_blending_mask=None, painted_mask=None):
    if paint_the_mask:
        modified_mask = painting_mask(input_mask)
        input_mask = torch.from_numpy(modified_mask).unsqueeze(0).cuda().long().clone().detach()
        vis_modified_mask = vis_seg(modified_mask)
        display_image_list([src_image, vis_modified_mask])
        painted_mask = input_mask

    if local_sketch:
        latent_local, local_blending_mask, visual_local_list = sketch_proxy(input_mask)
        display_image_list(visual_local_list)

    if global_cond is not None:
        assert isinstance(global_cond, str)
        latent_bald, visual_bald_list = bald_proxy(src_latent)
        display_image_list(visual_bald_list)
        print("bald image printed")

        if global_cond.endswith('.jpg') or global_cond.endswith('.png'):
            latent_global, visual_global_list = ref_proxy(global_cond, src_image, painted_mask=painted_mask)
        else:
            #
            latent_global, visual_global_list = text_proxy(global_cond, src_image, from_mean=True, painted_mask=painted_mask)
        display_image_list(visual_global_list)

    src_feature, edited_hairstyle_img = hairstyle_feature_blending(g_ema, seg, src_latent, src_feature, input_mask, latent_bald=latent_bald,\
                                                latent_global=latent_global, latent_local=latent_local, local_blending_mask=local_blending_mask)
    return src_feature, edited_hairstyle_img

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


Setting up Perceptual loss...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:01<00:00, 359MB/s]


Loading model from: /content/drive/MyDrive/2024_2_Basic_Study_HairStyler/criteria/lpips/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done


100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 98.5MiB/s]
Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth
100%|██████████| 85.7M/85.7M [00:07<00:00, 12.6MB/s]
Downloading: "https://www.adrianbulat.com/downloads/python-fan/3DFAN4-4a694010b9.zip" to /root/.cache/torch/hub/checkpoints/3DFAN4-4a694010b9.zip
100%|██████████| 91.9M/91.9M [00:09<00:00, 10.4MB/s]
Downloading: "https://www.adrianbulat.com/downloads/python-fan/depth-6c4283c0e0.zip" to /root/.cache/torch/hub/checkpoints/depth-6c4283c0e0.zip
100%|██████████| 224M/224M [00:20<00:00, 11.6MB/s]


Setting up Perceptual loss...
Loading model from: /content/drive/MyDrive/2024_2_Basic_Study_HairStyler/criteria/lpips/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done


In [14]:
!pip install streamlit --quiet
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 2s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [15]:
%%writefile app.py
import streamlit as st
import subprocess
import shutil
import os
import numpy as np
import io
from PIL import Image
from RAG_hairCLIP import get_hairStyleColor  # Import the function
from Hairclipv2 import hairstyle_editing_pipeline

# ✅ Define paths
UNALIGNED_PATH = "test_images/unaligned_img"
ALIGNED_PATH = "test_images/aligned_img"
SRC_PATH = "test_images/src_img"

# ✅ Ensure directories exist
os.makedirs(UNALIGNED_PATH, exist_ok=True)
os.makedirs(ALIGNED_PATH, exist_ok=True)
os.makedirs(SRC_PATH, exist_ok=True)

# ✅ Streamlit UI Setup
st.title("💇‍♀️ 헤어스타일 AI 어시스턴트")

# 🔸 **Section 1: Hair Style Suggestion**
st.subheader("🔍 헤어스타일 추천")
query = st.text_input("질문을 입력하세요:", placeholder="예: Suggest a hairstyle for a round face.")
if st.button("추천받기"):
    if query.strip():
        with st.spinner("AI가 헤어스타일을 추천하는 중..."):
            try:
                result = get_hairStyleColor(query)
                st.success("✅ 추천 완료!")
                st.text_area("추천 결과", value=result, height=200)
            except Exception as e:
                st.error(f"⚠️ 에러 발생: {str(e)}")
    else:
        st.warning("❗ 질문을 입력하세요!")

# 🔸 **Section 2: Hair Style Editing**
st.subheader("✂️ 헤어스타일 변경")
image_options = [
    "장원영.jpg",
    "심수민.jpg",
    "조현진.jpg",
    "김성재.jpg",
    "강민진.jpg",
    "차은우.jpg"
]
selected_image = st.selectbox("📸 사용할 이미지를 선택하세요:", image_options)
global_cond = st.text_input("원하는 헤어스타일을 입력하세요:", placeholder="예: bobbed hairstyle, curly hair")

unaligned_img_path = os.path.join(UNALIGNED_PATH, selected_image)
aligned_img_path = os.path.join(ALIGNED_PATH, selected_image)
src_img_path = os.path.join(SRC_PATH, selected_image)

# ✅ Show the original image
if os.path.exists(unaligned_img_path):
    st.image(unaligned_img_path, caption="원본 이미지", use_container_width=True)
else:
    st.error(f"❌ 이미지 파일이 없습니다: {unaligned_img_path}")

# ✅ 헤어스타일 변경 실행 버튼
if st.button("헤어스타일 변경 실행"):
    if global_cond and selected_image:
        with st.spinner("🔄 얼굴 정렬 중..."):
            try:
                # Run face alignment script
                subprocess.run(
                    ["python", "scripts/align_face.py", "-image_name", selected_image],
                    check=True, capture_output=True, text=True
                )
                st.success("✅ 얼굴 정렬 완료!")
            except subprocess.CalledProcessError as e:
                st.error(f"⚠️ 얼굴 정렬 중 오류 발생! \n\n {e.stderr}")

        # ✅ Move the aligned image to src_img
        if os.path.exists(aligned_img_path):
            shutil.move(aligned_img_path, src_img_path)
            st.success(f"✅ 정렬된 이미지가 저장되었습니다: {src_img_path}")
        else:
            st.error("⚠️ 정렬된 이미지를 찾을 수 없습니다. align_face.py 실행을 확인하세요.")

        with st.spinner("AI가 헤어스타일을 변경하는 중..."):
            try:
                # ✅ Run hairstyle editing pipeline
                result = hairstyle_editing_pipeline(selected_image.split(".")[0], global_cond)

                # ✅ Convert NumPy array to PIL Image if necessary
                if isinstance(result, np.ndarray):
                    if result.dtype == np.float32 or result.max() <= 1.0:
                        result = (result * 255).astype(np.uint8)
                    if result.shape[-1] == 3:
                        result = Image.fromarray(result)
                    elif result.shape[0] == 3:
                        result = Image.fromarray(np.transpose(result, (1, 2, 0)))

                # ✅ Display the result
                st.subheader("✨ 변경된 헤어스타일")
                st.image(result, caption="AI가 생성한 헤어스타일", use_container_width=True)

                # ✅ Add download button
                img_bytes = io.BytesIO()
                result.save(img_bytes, format="PNG")
                img_bytes = img_bytes.getvalue()

                st.download_button(
                    label="📥 이미지 다운로드",
                    data=img_bytes,
                    file_name=f"{selected_image.split('.')[0]}_edited_hairstyle.png",
                    mime="image/png"
                )
            except Exception as e:
                st.error(f"⚠️ 에러 발생: {str(e)}")
    else:
        st.warning("❗ 헤어스타일을 입력하고 이미지를 선택하세요!")



Overwriting app.py


In [16]:
!pip install pyngrok

In [17]:
!kill -9 $(pgrep -f ngrok)

^C


In [18]:
from pyngrok import ngrok
!ngrok config add-authtoken 2UVrdounebjFrqkNclMfdRQBT8G_7CmAsnFVpVG4gbfkeKxJ7

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [19]:
!streamlit run /content/drive/MyDrive/2024_2_Basic_Study_HairStyler/app.py &>/dev/null&

In [20]:
url = ngrok.connect(addr='8501')
print(url)

NgrokTunnel: "https://0400-35-229-141-99.ngrok-free.app" -> "http://localhost:8501"
